In [ ]:
import requests
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import plotly.express as px

In [ ]:
base_url = os.getenv("IMMICH_BASE_URL")
token = os.getenv("IMMICH_TOKEN")
year = os.getenv("YEAR")
start_date = f"{year}-01-01"
end_date = f"{year}-12-31"

headers = {"x-api-key": token}
body = {
    "takenAfter": start_date,
    "takenBefore": end_date,
    "withExif": "true",
    "size": 1000
}
all_photos_found = False
page = 0
all_photos = []

In [ ]:
# Loop thru photos cuz you can only fetch 1000 at a time
while not all_photos_found:
    if page > 0:
        body["page"] = page

    response = requests.request("POST", f"{base_url}/api/search/metadata", headers=headers, data=body)
    if response.status_code == 200:
        photos = response.json()
        print(f"{photos['assets']['total']} photos found")
        all_photos += photos['assets']['items']

        next_page = photos['assets']['nextPage']
        if next_page:
            print(f"next page: {next_page}")
            page = int(next_page)
        else:
            all_photos_found = True
    else:
        print(f"Error: {response.status_code}: {response.text}")
        all_photos_found = True

In [ ]:
photos_with_metadata = [photo for photo in all_photos if photo['exifInfo']['latitude'] != None]

In [ ]:
df = pd.DataFrame(photos_with_metadata)

In [ ]:
exif_cols = pd.json_normalize(df.exifInfo)
df[exif_cols.columns] = exif_cols

In [ ]:
df.columns

In [ ]:
filtered_df = df[['fileCreatedAt', 'latitude', 'longitude', 'city', 'state', 'country']].iloc[::-1]

In [ ]:
filtered_df['prev_latitude'] = [np.nan] + list(filtered_df['latitude'])[:-1]
filtered_df['prev_longitude'] = [np.nan] + list(filtered_df['longitude'])[:-1]

In [ ]:
filtered_df

In [ ]:
filtered_df['distance_from_prev'] = filtered_df.apply(lambda point: geodesic((point['latitude'], point['longitude']), (point['prev_latitude'], point['prev_longitude'])).km if np.isfinite(point['prev_latitude']) else 0, axis=1)

In [ ]:
final_df = filtered_df[filtered_df['distance_from_prev'] > 70]

In [ ]:
final_df

In [ ]:
unique_cities = final_df.groupby(['city', 'state']).ngroups
unique_states = final_df['state'].nunique()
unique_countries = final_df['country'].nunique()
total_distance_traveled = round(final_df['distance_from_prev'].sum(), 2)
most_common_state = final_df['state'].mode()

In [ ]:
print(f"{year} Stats")
print(f"{unique_cities} unique cities")
print(f"{unique_states} unique states")
print(f"{unique_countries} unique countries")
print(f"{total_distance_traveled}km total distance traveled")
print(f"{most_common_state} is the most visited state")

In [ ]:
# Create plain text date col
final_df['date_string'] = pd.to_datetime(final_df['fileCreatedAt'])
final_df['date_string'] = final_df['date_string'].dt.strftime("%B %Y")

In [ ]:
final_df[['latitude', 'longitude', 'city', 'state', 'country', 'date_string']].to_csv("export.csv", index=False)

In [ ]:
i = 0
group = 0
animated_json = {'latitude': [], 'longitude': [], 'fileCreatedAt': [], 'city': [], 'state': [], 'country': [], 'group': []}
for index, row in final_df.iterrows():
    animated_json['latitude'].append(row['latitude'])
    animated_json['longitude'].append(row['longitude'])
    animated_json['fileCreatedAt'].append(row['fileCreatedAt'])
    animated_json['city'].append(row['city'])
    animated_json['state'].append(row['state'])
    animated_json['country'].append(row['country'])
    animated_json['group'].append(group)

    if i > 0 and i < len(final_df) - 1:
        group+=1
        animated_json['latitude'].append(row['latitude'])
        animated_json['longitude'].append(row['longitude'])
        animated_json['fileCreatedAt'].append(row['fileCreatedAt'])
        animated_json['city'].append(row['city'])
        animated_json['state'].append(row['state'])
        animated_json['country'].append(row['country'])
        animated_json['group'].append(group)

    i+=1
    


In [ ]:
animated_df = pd.DataFrame(animated_json)

In [ ]:
fig = px.line_map(animated_df,
            lat='latitude',
            lon='longitude',
            animation_frame='group',
            zoom=4,
            width=1200,
            height=800)

In [ ]:
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()